## Vessels

In [20]:
# create vessel class
class vessel_properties_mixin(object):
    def __init__(self, 
                 vessel_type, call_size, LOA, draft, beam, max_cranes, all_turn_time, mooring_time, demurrage, 
                 *args, **kwargs):
        super().__init__(*args, **kwargs)
        "initialize"
        self.vessel_type   = vessel_type
        self.call_size     = call_size 
        self.LOA           = LOA
        self.draft         = draft
        self.beam          = beam
        self.max_cranes    = max_cranes
        self.all_turn_time = all_turn_time
        self.mooring_time  = mooring_time
        self.demurrage     = demurrage
    
    
# Since this is presumably not very variable input it can be standardized as followed

handysize_data = {"vessel_type": 'Handysize', "call_size": 35000, 
                  "LOA": 130, "draft": 10, "beam": 24, "max_cranes": 2, 
                  "all_turn_time": 24, "mooring_time": 3, "demurrage": 600} 
handymax_data = {"vessel_type": 'Handymax', "call_size": 50000, 
                  "LOA": 180, "draft": 11.5, "beam": 28, "max_cranes": 2, 
                  "all_turn_time": 24, "mooring_time": 3, "demurrage": 750} 
panamax_data = {"vessel_type": 'Panamax', "call_size": 65000, 
                  "LOA": 220, "draft": 13, "beam": 32.2, "max_cranes": 3, 
                  "all_turn_time": 36, "mooring_time": 4, "demurrage": 730} 

In [21]:
# create vessel class
class vessel(vessel_properties_mixin):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

In [23]:
# here I created a JSON to prepare the inputs. 
# Since this is presumably not very variable input it can be standardized like this.

# create input


# create objects
handysize = vessel(**handysize_data)
handymax = vessel(**handymax_data)
panamax = vessel(**panamax_data)

# print object dicts
# print(handysize.__dict__)
# print(handymax.__dict__)
# print(panamax.__dict__)

## Quay wall

In [13]:
# create quay wall class
class quay_wall(object):
    def __init__(self, ownership, lifespan, unit_rate, mobilisation, mobilisation_perc, maintenance, length, depth, *args, **kwargs):
        super().__init__(*args, **kwargs)
        "initialize"
        self.ownership         = ownership
        self.lifespan          = lifespan
        self.unit_rate         = unit_rate
        self.mobilisation      = mobilisation
        self.mobilisation_perc = mobilisation_perc
        self.maintenance       = maintenance
        self.length            = length
        self.depth             = depth

## Cranes

In [14]:
# create cyclic loader class
class cyclic_properties_mixin(object):
    def __init__(self, ownership, lifespan, unit_rate, mobilisation, maintenance, consumption, crew, unloader_type, lifting_capacity, hourly_cycles, eff_fact, quantity, *args, **kwargs):
        super().__init__(*args, **kwargs)
        "initialize"
        self.ownership          = ownership
        self.lifespan           = lifespan
        self.unit_rate          = unit_rate
        self.mobilisation       = mobilisation
        self.maintenance        = maintenance
        self.consumption        = consumption
        self.crew               = crew
        self.unloader_type      = unloader_type
        self.lifting_capacity   = lifting_capacity
        self.hourly_cycles      = hourly_cycles
        self.eff_fact           = eff_fact 
        self.payload            = 0.70 * self.lifting_capacity      #Source: Nemag
        self.peak_capacity      = self.payload * self.hourly_cycles #Or as direct input 
        self.effective_capacity = eff_fact * self.peak_capacity     #Source: TATA steel
        self.quantity           = quantity
        
        
# create continuous loader class
class continuous_properties_mixin(object):
    def __init__(self, ownership, lifespan, unit_rate, mobilisation, maintenance, consumption, crew, unloader_type, peak_capacity, eff_fact, quantity, *args, **kwargs):
        super().__init__(*args, **kwargs)
        "initialize"
        self.ownership          = ownership
        self.lifespan           = lifespan
        self.unit_rate          = unit_rate
        self.mobilisation       = mobilisation
        self.maintenance        = maintenance
        self.consumption        = consumption
        self.crew               = crew
        self.unloader_type      = unloader_type
        self.peak_capacity      = peak_capacity
        self.eff_fact           = eff_fact 
        self.effective_capacity = eff_fact * self.peak_capacity     #Source: TATA steel
        self.quantity           = quantity     

## Conveyors

In [15]:
# create quay wall class
class conveyor_properties_mixin(object):
    def __init__(self, ownership, lifespan, unit_rate, mobilisation, maintenance, consumption, crew, length, quantity, *args, **kwargs):
        super().__init__(*args, **kwargs)
        "initialize"
        self.ownership         = ownership
        self.lifespan          = lifespan
        self.unit_rate         = unit_rate
        self.mobilisation      = mobilisation
        self.maintenance       = maintenance
        self.consumption       = consumption
        self.crew              = crew
        self.length            = length
        self.quantity          = quantity

## Storage

In [16]:
# create storage class
class storage_properties_mixin(object):
    def __init__(self, ownership, lifespan, unit_rate, mobilisation, mobilisation_perc, maintenance, consumption, crew, storage_type, capacity, *args, **kwargs):
        super().__init__(*args, **kwargs)
        "initialize"
        self.ownership         = ownership
        self.lifespan          = lifespan
        self.unit_rate         = unit_rate
        self.mobilisation      = mobilisation
        self.mobilisation_perc = mobilisation_perc
        self.maintenance       = maintenance
        self.consumption       = consumption
        self.crew              = crew
        self.storage_type      = storage_type
        self.capacity          = capacity

## Loading Station

In [17]:
# create loading station class
class loading_station_properties_mixin(object):
    def __init__(self, ownership, lifespan, unit_rate, mobilisation, maintenance, consumption, crew, capacity, *args, **kwargs):
        super().__init__(*args, **kwargs)
        "initialize"
        self.ownership         = ownership
        self.lifespan          = lifespan
        self.unit_rate         = unit_rate
        self.mobilisation      = mobilisation
        self.maintenance       = maintenance
        self.consumption       = consumption
        self.crew              = crew
        self.capacity          = capacity

In [ ]:
class 